In [1]:
import pandas as pd
import gzip
import os
import numpy as np

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/")
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [3]:
arch_sellout = "datasets/tb_sellout_02.txt.gz"
arch_ejemplo_prediccion = "datasets/tb_kaggle_muestra.csv"
arch_salida_prefijo = "datasets/productos_a_predecir"

In [4]:
# Variables para definir que atributos se descartan
meses_para_control_vigencia = [201904,201905,201906] #meses en los cuales deben aparecer los productos para ser considerados vigentes (NO discontinuados) y ser tomados en la prediccion
tope_fecha_historia = 201902 #los productos que aparezcan desde este mes (inclusive) en adelante, se excluyen por tener poca historia de ventas

## Sellout

In [5]:
# Abrir el archivo .gz y cargarlo en un DataFrame
with gzip.open(arch_sellout, 'rt') as archivo:
    # Leer el archivo línea por línea
    df_sellout = pd.read_csv(archivo,sep="\t")

In [6]:
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 157.3 MB


In [7]:
# Por las dudas, eliminamos duplicados
print(len(df_sellout))
df_sellout.drop_duplicates(inplace=True)
print(len(df_sellout))

2945818
2945818


**--> sin duplicados**

In [8]:
df_sellout.isna().sum()

periodo                  0
customer_id              0
product_id               0
plan_precios_cuidados    0
cust_request_qty         0
cust_request_tn          0
tn                       0
dtype: int64

**--> sin nulos**

In [9]:
df_sellout.periodo.unique()                

array([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708,
       201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804,
       201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812,
       201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912])

In [10]:
len(df_sellout.product_id.unique())          

1233

--> algunos productos no van a tener descripción

In [11]:
len(df_sellout.customer_id.unique())            

597

In [12]:
df_sellout.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


## Descarte de Productos que no hay que predecir

In [13]:
# Discontinuados (sin ventas en 3 meses mas adelante)
product_ids_vigentes =  df_sellout[df_sellout.periodo.isin(meses_para_control_vigencia)].product_id.unique()
print("Vigentes:", len(product_ids_vigentes))

product_ids_discontinuados = set(df_sellout.product_id.unique()).difference(set(product_ids_vigentes))
print("Discontinuados:", len(product_ids_discontinuados))

#Sin historia suficiente (minima fecha >= 201902)
product_ids_sin_hist_sufic = df_sellout.groupby("product_id").agg({"periodo":"min"}).reset_index()
product_ids_sin_hist_sufic = product_ids_sin_hist_sufic[product_ids_sin_hist_sufic.periodo>=tope_fecha_historia]
print(product_ids_sin_hist_sufic.head(5))
product_ids_sin_hist_sufic = product_ids_sin_hist_sufic.product_id.unique()
print("Menos 3 meses hist:", len(product_ids_sin_hist_sufic))

#Interseccion y union
print("Interseccion: ", len((set(product_ids_discontinuados).intersection(set(product_ids_sin_hist_sufic)))))
print("Union: ", len((set(product_ids_discontinuados).union(set(product_ids_sin_hist_sufic)))))


product_ids_para_predecir = set(df_sellout.product_id.unique()).difference((set(product_ids_discontinuados).union(set(product_ids_sin_hist_sufic))))
print("\nTotal:", len(product_ids_para_predecir))

Vigentes: 958
Discontinuados: 275
     product_id  periodo
31        20032   201902
126       20127   201909
173       20174   201906
208       20210   201909
211       20213   201908
Menos 3 meses hist: 182
Interseccion:  92
Union:  365

Total: 868


In [14]:
#Se guardan en un csv aparte
df_prods_prediccion = pd.DataFrame(data={"product_id":list(product_ids_para_predecir)})
df_prods_prediccion.to_csv(arch_salida_prefijo + "_" + str(tope_fecha_historia) + ".csv", index=False)
df_prods_prediccion.head()

,product_id
0,20480
1,20481
2,20482
3,20483
4,20484


## Creación de Productos Predicción Final (201912)

In [15]:
df_ejemplo_prediccion = pd.read_csv(arch_ejemplo_prediccion)
df_ejemplo_prediccion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   product_id  780 non-null    int64  
 1   tn          780 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 12.3 KB


In [16]:
df_ejemplo_prediccion.head()

,product_id,tn
0,20001,1504.68856
1,20002,1087.30855
2,20003,892.50129
3,20004,637.90002
4,20005,593.24443


In [17]:
df_prods_prediccion = df_ejemplo_prediccion[["product_id"]]
df_prods_prediccion.to_csv(arch_salida_prefijo + "_201912.csv", index=False)
df_prods_prediccion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   product_id  780 non-null    int64
dtypes: int64(1)
memory usage: 6.2 KB


In [18]:
# Se esperan 780
len(df_prods_prediccion)

780